# Explore here

In [272]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


url = "https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv"

# Specify the separator as ';'
df = pd.read_csv(url, sep=';')

df.to_csv('bank_data.csv', index=False)

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [273]:
#Check the number of rows and columns
df.shape

(41188, 21)

In [274]:
#Check the type of variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [275]:
#Check for constant values
df.nunique()

#There are no constant values

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [276]:
#check for duplicates
df.duplicated().sum()

np.int64(12)

In [277]:
#Since we have duplicate entries in the dataset, we should eliminate them to avoid redundancy in our algorithm
df = df.drop_duplicates()
df = df.reset_index(drop=True)

df.duplicated().sum()

np.int64(0)

In [278]:
#Basic column description
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41176.0,40.023800,10.420680,17.000,32.000,38.000,47.000,98.000
duration,41176.0,258.315815,259.305321,0.000,102.000,180.000,319.000,4918.000
campaign,41176.0,2.567879,2.770318,1.000,1.000,2.000,3.000,56.000
pdays,41176.0,962.464810,186.937102,0.000,999.000,999.000,999.000,999.000
previous,41176.0,0.173013,0.494964,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41176.0,0.081922,1.570883,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41176.0,93.575720,0.578839,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41176.0,-40.502863,4.627860,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41176.0,3.621293,1.734437,0.634,1.344,4.857,4.961,5.045
nr.employed,41176.0,5167.034870,72.251364,4963.600,5099.100,5191.000,5228.100,5228.100


In [279]:
#Check for empty elements in the dataset
df.isnull().mean()*100

#There are no missing elements

age               0.0
job               0.0
marital           0.0
education         0.0
default           0.0
housing           0.0
loan              0.0
contact           0.0
month             0.0
day_of_week       0.0
duration          0.0
campaign          0.0
pdays             0.0
previous          0.0
poutcome          0.0
emp.var.rate      0.0
cons.price.idx    0.0
cons.conf.idx     0.0
euribor3m         0.0
nr.employed       0.0
y                 0.0
dtype: float64

Variable scaling

In [280]:
#Despite not having any missing values, many variables have values marked as 'unknown' or 'nonexistent'. This values should be treated as missing values.
#We first detect the percentage of missing values in each column
unknown_default_percentage = (df['default'] == 'unknown').mean() * 100
unknown_loan_percentage = (df['loan'] == 'unknown').mean() * 100
unknown_housing_percentage = (df['housing'] == 'unknown').mean() * 100
nonexistent_poutcome_percentage = (df['poutcome'] == 'nonexistent').mean() * 100
unknown_marital_percentage = (df['marital'] == 'unknown').mean() * 100
unknown_education_percentage = (df['education'] == 'unknown').mean() * 100

print(f'default: {unknown_default_percentage}%, housing: {unknown_housing_percentage}%, loan: {unknown_loan_percentage}, poutcome: {nonexistent_poutcome_percentage}%, marital: {unknown_marital_percentage}%, education: {unknown_education_percentage}%')

default: 20.876238585583835%, housing: 2.4043131921507674%, loan: 2.4043131921507674, poutcome: 86.33912959005245%, marital: 0.19428793471925393%, education: 4.2014765883038665%


In [281]:
#default and poutcome have more than 20% of unknown values (86% in the case of poutcome) and thus should be removed from our dataset
df.drop(['default', 'poutcome'], axis=1, inplace=True)

#The other variables have less than 5% of missing values. Therefore, we will remove the rows with 'unknown' or 'nonexistent' values
df_cleaned = df[~df.apply(lambda row: row.str.contains('unknown', case=False).any(), axis=1)].reset_index(drop=True)

#We will now convert the rest of values to numerical

#Binary variables
df_cleaned['y'] = df_cleaned['y'].map({'yes': 1, 'no': 0})
df_cleaned['contact'] = df_cleaned['contact'].map({'telephone': 0, 'cellular': 1})
df_cleaned['housing'] = df_cleaned['housing'].map({'yes': 1, 'no': 0})
df_cleaned['loan'] = df_cleaned['loan'].map({'yes': 1, 'no': 0})

#Weekdays and months
df_cleaned['month'] = df_cleaned['month'].map({'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12})
df_cleaned['day_of_week'] = df_cleaned['day_of_week'].map({'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5})

#Rest of categorical variables (job, marital and education)
df_cleaned['job'] = pd.factorize(df_cleaned['job'])[0]
df_cleaned['marital'] = df_cleaned['marital'].map({'single': 0, 'married': 1, 'divorced': 2})
df_cleaned['education'] = df_cleaned['education'].map({'illiterate': 0, 'basic.4y': 1, 'basic.6y': 2, 'basic.9y': 3, 'high.school': 4, 'professional.course': 5, 'university.degree': 6})

df_cleaned.info()

#All of our variables are numerical now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38234 entries, 0 to 38233
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             38234 non-null  int64  
 1   job             38234 non-null  int64  
 2   marital         38234 non-null  int64  
 3   education       38234 non-null  int64  
 4   housing         38234 non-null  int64  
 5   loan            38234 non-null  int64  
 6   contact         38234 non-null  int64  
 7   month           38234 non-null  int64  
 8   day_of_week     38234 non-null  int64  
 9   duration        38234 non-null  int64  
 10  campaign        38234 non-null  int64  
 11  pdays           38234 non-null  int64  
 12  previous        38234 non-null  int64  
 13  emp.var.rate    38234 non-null  float64
 14  cons.price.idx  38234 non-null  float64
 15  cons.conf.idx   38234 non-null  float64
 16  euribor3m       38234 non-null  float64
 17  nr.employed     38234 non-null 

In [282]:
#Data split
X = df_cleaned.drop(['y'], axis=1)
y = df_cleaned['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [283]:
#Visualize the importance of each observation on loan
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42).fit(X_train, y_train)
df_imp = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb_model.feature_importances_ * 100})
df_imp = df_imp.sort_values(by='Importance', ascending=False)
df_imp

,Feature,Importance
17,nr.employed,69.662254
9,duration,5.595479
13,emp.var.rate,4.039667
7,month,3.927958
11,pdays,3.210233
16,euribor3m,1.977944
6,contact,1.531636
14,cons.price.idx,1.260463
12,previous,1.050031
15,cons.conf.idx,0.993290


In [284]:
#Since the amount of variables isn't very big, we will remove those that had an importance of less than 1%
X_train = X_train[df_imp[df_imp.Importance > 1]['Feature'].values]
X_test = X_test[df_imp[df_imp.Importance > 1]['Feature'].values]

#Show correlation between variables
X_corr = X_train.copy()
X_corr['y'] = y_train
corr = X_corr.corr()
corr.style.background_gradient(cmap='coolwarm').format(precision=3)

,nr.employed,duration,emp.var.rate,month,pdays,euribor3m,contact,cons.price.idx,previous,y
nr.employed,1.000,-0.040,0.908,0.143,0.365,0.945,-0.267,0.522,-0.497,-0.342
duration,-0.040,1.000,-0.023,-0.022,-0.046,-0.028,0.024,0.012,0.016,0.406
emp.var.rate,0.908,-0.023,1.000,0.068,0.272,0.972,-0.392,0.774,-0.427,-0.287
month,0.143,-0.022,0.068,1.000,-0.079,0.174,0.322,-0.146,0.059,0.038
pdays,0.365,-0.046,0.272,-0.079,1.000,0.293,-0.117,0.086,-0.575,-0.318
euribor3m,0.945,-0.028,0.972,0.174,0.293,1.000,-0.397,0.688,-0.456,-0.294
contact,-0.267,0.024,-0.392,0.322,-0.117,-0.397,1.000,-0.594,0.212,0.137
cons.price.idx,0.522,0.012,0.774,-0.146,0.086,0.688,-0.594,1.000,-0.218,-0.127
previous,-0.497,0.016,-0.427,0.059,-0.575,-0.456,0.212,-0.218,1.000,0.217
y,-0.342,0.406,-0.287,0.038,-0.318,-0.294,0.137,-0.127,0.217,1.000


In [285]:
#VIF calculation
# vif = pd.Series([variance_inflation_factor(X_train.corr().values, i) for i in range(X_train.corr().shape[1])], index=X_train.columns) 
# vif

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

calculate_vif(X_train.corr())

,Feature,VIF
0,nr.employed,2211.459597
1,duration,1.241902
2,emp.var.rate,14123.866800
3,month,79.026661
4,pdays,6.777847
5,euribor3m,19420.394732
6,contact,160.593796
7,cons.price.idx,524.571521
8,previous,9.513853


In [286]:
#We can see that several of our variables are highly correlated. This can skew our database, so it's best to elliminate those variables with high correlation 
while True:
    vif = calculate_vif(X_train.corr())
    max_vif = vif["VIF"].max()
    if max_vif > 10:
        feature_to_remove = vif.loc[vif["VIF"].idxmax(), "Feature"]
        print(f"Removing feature {feature_to_remove} with VIF={max_vif}")
        X_train = X_train.drop(columns=[feature_to_remove])
    else:
        break

calculate_vif(X_train.corr())

Removing feature euribor3m with VIF=19420.39473195123
Removing feature emp.var.rate with VIF=1954.9848367122208
Removing feature cons.price.idx with VIF=10.743078446054485


,Feature,VIF
0,nr.employed,4.410755
1,duration,1.019403
2,month,2.139186
3,pdays,4.499938
4,contact,2.437551
5,previous,6.636426


In [287]:
X_corr = X_train.copy()
X_corr['y'] = y_train
corr = X_corr.corr()
corr.style.background_gradient(cmap='coolwarm').format(precision=3)

,nr.employed,duration,month,pdays,contact,previous,y
nr.employed,1.000,-0.040,0.143,0.365,-0.267,-0.497,-0.342
duration,-0.040,1.000,-0.022,-0.046,0.024,0.016,0.406
month,0.143,-0.022,1.000,-0.079,0.322,0.059,0.038
pdays,0.365,-0.046,-0.079,1.000,-0.117,-0.575,-0.318
contact,-0.267,0.024,0.322,-0.117,1.000,0.212,0.137
previous,-0.497,0.016,0.059,-0.575,0.212,1.000,0.217
y,-0.342,0.406,0.038,-0.318,0.137,0.217,1.000


In [288]:
#Create our logistic model
model = LogisticRegression()
model.fit(X_train,y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [289]:
#Train our model and make a prediction
#We first delete the variables 'cons.price.idx', 'emp.var.rate', 'euribor3m' from X_test, so that it matches X_train
X_test = X_test.drop(['cons.price.idx', 'emp.var.rate', 'euribor3m'], axis = 1) 

y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(7647,))

In [290]:
accuracy = accuracy_score(y_test,y_pred)

print(f'After testing our model, we can predict variable y with an accuracy of {accuracy * 100}% ')

After testing our model, we can predict variable y with an accuracy of 90.50608081600627% 
